In [74]:
!pip install django

from django.shortcuts import render, HttpResponse
import numpy as np
import pandas as pd
import os
from nlp.settings import BASE_DIR
import json


# Create your views here.
from nlp_proj.models import Hotel

def index(request):
    txtfile = pd.read_csv(os.path.join(BASE_DIR, 'nlp_proj/dummydata/dummy.txt') , sep=",", names=["txt", "category", "label"])
    overall = txtfile.groupby("label").count().iloc[:3, 1].values.tolist()
    overall_keys = ["negative", "positive"]
    overall_dic = {i : j for i, j in zip(overall_keys, overall)}
    overall_json = json.dumps(overall_dic)

    #sents8_json = pd.read_csv(os.path.join(BASE_DIR, 'nlp_proj/dummydata/FastText_8_sentiment_results.txt') , sep=",", names=["txt", "category", "sentiment", "rawIdx"])
    #overall = txtfile.groupby("label").count().iloc[:3, 1].values
    #overall_dic = {"neg" : overall[0], "pos" : overall[1]}
    #overall_json = json.dumps(overall_dic)

    #sents17_json = pd.read_csv(os.path.join(BASE_DIR, 'nlp_proj/dummydata/FastText_17_sentiment_results.txt') , sep=",", names=["txt", "category", "sentiment", "rawIdx"])


    # review_all = Hotel.objects.all()  # .get(), .filter(), ...
    # request가 POST -> Form을 완성.
    # Form이 유효하면 저장.\
    # if request.method == "POST":
    #     form = CoffeeForm(request.POST)  # 완성된 Form
    #     if form.is_valid():  # 채워진 Form이 유효하다면
    #         form.save()  # Form을 Model에 저장

    # form = CoffeeForm()
    return render(request, 'index.html', {"overall_json" : overall_json})

ModuleNotFoundError: No module named 'nlp'

In [76]:
txtfile = pd.read_csv('dummydata/dummy.txt', sep=",", names=["txt", "category", "label"])
overall = txtfile.groupby("label").count().iloc[:3, 1].values.tolist()
overall_keys = ["negative", "positive"]
overall_dic = {i : j for i, j in zip(overall_keys, overall)}
overall_dic


{'negative': 517, 'positive': 350}

In [96]:
txtfile = pd.read_csv('dummydata/FastText_8_sentiment_results.txt', sep=",")
txtfile


,text,category,sentiment,rawsentenceIdx
0,실내 수영장도 개방형이라서 좋았다,깨끗,1,3
1,객실 인테리어 및 편의시설은 훌륭했다,깨끗,1,3
2,란 생각이 마구 들더군요,깨끗,0,4
3,신라는 반얀에 비해 층 고도 높고 룸도 쾌적하고 시설도 더 깔끔하고 조식이 훨씬 맛...,깨끗,1,4
4,3 청결 서비스 숙소 내 청결도는 상당히 만족했습니다,깨끗,1,6
...,...,...,...,...
5035,아기랑 편하게 체크인 아웃할 수 있게 배려해 주셔서 감사했습니다,기타,1,1623
5036,조식 맛있지만 사람이 많아서 계속 줄 줄줄 ㅠㅠ 실내 수영장은 평일은 아기 출입금지...,기타,0,1623
5037,아기랑 갈분은 참고하 세용 하튼 한 번은 가 볼 만한 특고급 호텔 잘 다녀왔습니당,기타,1,1623
5038,체크인 시간보다 일찍 갔더니 바로 체크인 되지는 않았고 연락처로 체크인 가능할 때 ...,기타,0,1626


In [97]:
category_pos_cnt = txtfile.groupby("category").sum()
category_pos_cnt

,sentiment,rawsentenceIdx
category,,
기타,1207,2037838
깨끗,699,748467
소리,4,18462
위치,217,353758
조식,643,869081
직원,528,561636
화장실,118,247201


In [98]:
category_total_cnt = txtfile.groupby("category").count()
category_total_cnt

,text,sentiment,rawsentenceIdx
category,,,
기타,2072,2072,2072
깨끗,842,842,842
소리,21,21,21
위치,306,306,306
조식,884,884,884
직원,650,650,650
화장실,265,265,265


In [99]:
category_total_cnt["pos_cnt"] = category_pos_cnt["sentiment"]
category_total_cnt["neg_cnt"] = category_total_cnt["sentiment"] - category_total_cnt["pos_cnt"]

In [100]:
category_total_cnt

,text,sentiment,rawsentenceIdx,pos_cnt,neg_cnt
category,,,,,
기타,2072,2072,2072,1207,865
깨끗,842,842,842,699,143
소리,21,21,21,4,17
위치,306,306,306,217,89
조식,884,884,884,643,241
직원,650,650,650,528,122
화장실,265,265,265,118,147


In [101]:
import math

category_total_cnt["pos_ratio"] = category_total_cnt["pos_cnt"] / (category_total_cnt["pos_cnt"] + category_total_cnt["neg_cnt"])
category_total_cnt["neg_ratio"] = category_total_cnt["neg_cnt"] / (category_total_cnt["pos_cnt"] + category_total_cnt["neg_cnt"])
category_total_cnt


category_total_cnt["pos_cnt_sqrt"] = category_total_cnt["pos_cnt"].apply(math.sqrt)
category_total_cnt["neg_cnt_sqrt"] = category_total_cnt["neg_cnt"].apply(math.sqrt)

In [102]:
category_total_cnt["pos_score"] = category_total_cnt["pos_cnt_sqrt"] + category_total_cnt["pos_ratio"]
category_total_cnt["neg_score"] = category_total_cnt["neg_cnt_sqrt"] + category_total_cnt["neg_ratio"]
category_total_cnt

,text,sentiment,rawsentenceIdx,pos_cnt,neg_cnt,pos_ratio,neg_ratio,pos_cnt_sqrt,neg_cnt_sqrt,pos_score,neg_score
category,,,,,,,,,,,
기타,2072,2072,2072,1207,865,0.582529,0.417471,34.741906,29.410882,35.324434,29.828353
깨끗,842,842,842,699,143,0.830166,0.169834,26.438608,11.958261,27.268774,12.128094
소리,21,21,21,4,17,0.190476,0.809524,2.000000,4.123106,2.190476,4.932629
위치,306,306,306,217,89,0.709150,0.290850,14.730920,9.433981,15.440070,9.724831
조식,884,884,884,643,241,0.727376,0.272624,25.357445,15.524175,26.084820,15.796799
직원,650,650,650,528,122,0.812308,0.187692,22.978251,11.045361,23.790558,11.233053
화장실,265,265,265,118,147,0.445283,0.554717,10.862780,12.124356,11.308064,12.679073


In [111]:
category_total_cnt["pos_score"]

category
기타     35.324434
깨끗     27.268774
소리      2.190476
위치     15.440070
조식     26.084820
직원     23.790558
화장실    11.308064
Name: pos_score, dtype: float64

KeyError: '기타'

In [103]:
total_cnt = {"positive" : category_total_cnt["pos_cnt"].sum() , "negative" : category_total_cnt["neg_cnt"].sum()}
total_cnt

{'positive': 3416, 'negative': 1624}

In [104]:
txtfile = pd.read_csv('dummydata/dummy.txt', sep=",", names=["txt", "category", "label"])
print(txtfile.groupby("label").count().iloc[:3, 1])
overall = txtfile.groupby("label").count().iloc[:3, 1].values.tolist()
print(overall)
overall_keys = ["negative", "positive"]
overall_dic = {i : j for i, j in zip(overall_keys, overall)}
print(overall_dic)
#overall_json = json.dumps(overall_dic)

label
0    517
1    350
Name: category, dtype: int64
[517, 350]
{'negative': 517, 'positive': 350}
